#### Prepare 80 random bytes for small testing purpose

In [6]:
x = rand(UInt8, 8 * 10);

#### Define a function that converts 8 bytes into a single 64-bit unsigned integer

In [7]:
f(a,b,c,d,e,f,g,h) = (UInt64(a) << 56) | (UInt64(b) << 48) | 
    (UInt64(c) << 40) | (UInt64(d) << 32) | 
    (UInt64(e) << 24) | (UInt64(f) << 16) | 
    (UInt64(g) << 8) | UInt64(h)

f (generic function with 1 method)

In [8]:
f(x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8])

0xef91beb5eb863d46

In [9]:
x[1:8]'

1×8 RowVector{UInt8,Array{UInt8,1}}:
 0xef  0x91  0xbe  0xb5  0xeb  0x86  0x3d  0x46

#### Cool!  Let's apply the function evenly at 8-byte intervals

In [5]:
f.(x[1:8:end],x[2:8:end],x[3:8:end],x[4:8:end],
    x[5:8:end], x[6:8:end], x[7:8:end], x[8:8:end])

10-element Array{UInt64,1}:
 0x77060c5e5d716cac
 0x9cac05468f5d6e1f
 0x0a307086c0144809
 0x622368fae9938005
 0xd4b10d1d565e756c
 0xa72fc07832c524f4
 0x166fe76b7f61a4b5
 0x85004f6a1a9618d2
 0xa463de4d3accf813
 0xdb62335d7834ea9d

#### Now, let's create a much larger sample with 80,000 bytes

In [15]:
N = 10000
x = rand(UInt8, 8 * N);

In [16]:
length(x)

80000

In [17]:
display(x[1:8]')
display(x[end-7:end]')

1×8 RowVector{UInt8,Array{UInt8,1}}:
 0xe4  0x26  0x9a  0xef  0xd5  0xcd  0xe3  0x25

1×8 RowVector{UInt8,Array{UInt8,1}}:
 0xe2  0x64  0xaa  0xa3  0x20  0xc4  0x1c  0xe5

#### Moment of truth!  Let's do some benchmarks.

In [19]:
using BenchmarkTools

##### Method 1 - vectorized function call

In [24]:
y = @btime f.(x[1:8:end],x[2:8:end],x[3:8:end],x[4:8:end],
    x[5:8:end], x[6:8:end], x[7:8:end], x[8:8:end]);
y[1:3]

  231.939 μs (157 allocations: 161.95 KiB)


3-element Array{UInt64,1}:
 0xe4269aefd5cde325
 0x3e8ea3affc7a7227
 0x95e0716e61b4b55b

##### Method 2 - allocate an array and run a for-loop using reinterpret
This takes longer and use more memory.

In [48]:
A = Array{UInt64}(N)
function g(x, N, A) 
    z = 1
    for i in 1:8:N*8
        A[z] = bswap.(reinterpret(UInt64, x[i:i+8-1]))[1]
        z += 1
    end
end
@btime g(x, N, A)
A[1:3]

  1.085 ms (40000 allocations: 2.59 MiB)


3-element Array{UInt64,1}:
 0xe4269aefd5cde325
 0x3e8ea3affc7a7227
 0x95e0716e61b4b55b

#### Let's make an even bigger test case

In [59]:
N = 100_000
x = rand(UInt8, 8 * N);

#### Method 1 - use array comprehension to slice the byte array and interpret

In [60]:
function convertfloat(bytes)
    values = [bytes[i:i+8-1] for i in 1:8:length(bytes)]
    convertedvalues = map(x -> reinterpret(Float64, x)[1], values)
    convertedvalues
end
y = @btime convertfloat(x);
y[1:3]

  9.591 ms (300007 allocations: 18.31 MiB)


3-element Array{Float64,1}:
  6.88627e-241
 -9.96849e-88 
  1.85544e-14 

#### Method 2 - create a simple reinterpret function and use vectorized call 

In [61]:
reinterpretF64(v) = reinterpret(Float64, v)[1]
function convertfloat(bytes)
    values = [bytes[i:i+8-1] for i in 1:8:length(bytes)]
    reinterpretF64.(values)
end
y = @btime convertfloat(x);
y[1:3]

  9.608 ms (300032 allocations: 18.31 MiB)


3-element Array{Float64,1}:
  6.88627e-241
 -9.96849e-88 
  1.85544e-14 

#### More serious performance comparison 

In [65]:
# Version a.  Original implementation... slow.
"""
Byte swap is needed only if file the array represent a different endianness
than the system.  This function does not make any assumption and the caller
is expected to pass `true` to the `swap` argument when needed.
"""
function convertfloat64a(bytes::Vector{UInt8}, swap::Bool)
    values = [bytes[i:i+8-1] for i in 1:8:length(bytes)]
    values = map(x -> reinterpret(Float64, x)[1], values)
    swap ? bswap.(values) : values
end

# Version b.  Should be a lot faster.  
"""
It turns out that `reinterpret` consider a single UInt64 as BigEndian 
Hence it's necessary to swap bytes if the array is in LittleEndian convention.
This function does not make any assumption and the caller
is expected to pass `true` to the `swap` argument when needed.
"""
function convertfloat64b(bytes::Vector{UInt8}, endianess::Symbol) 
    v = endianess == :LittleEndian ? reverse(bytes) : bytes
    c = convertint64.(v[1:8:end],v[2:8:end],v[3:8:end],v[4:8:end],
            v[5:8:end], v[6:8:end], v[7:8:end], v[8:8:end])
    r = reinterpret.(Float64, c)
    endianess == :LittleEndian ? reverse(r) : r
end

"""
Take 8 bytes and convert them into a UInt64 type.  The order is preserved.
"""
function convertint64(a::UInt8,b::UInt8,c::UInt8,d::UInt8,e::UInt8,f::UInt8,g::UInt8,h::UInt8)
    (UInt64(a) << 56) | (UInt64(b) << 48) | 
    (UInt64(c) << 40) | (UInt64(d) << 32) | 
    (UInt64(e) << 24) | (UInt64(f) << 16) | 
    (UInt64(g) << 8)  |  UInt64(h)
end

convertint64

In [74]:
@time z1 = convertfloat64a(x, false)
display(z1[1:3])
display(z1[end-2:end])
size(z1)

3-element Array{Float64,1}:
  6.88627e-241
 -9.96849e-88 
  1.85544e-14 

3-element Array{Float64,1}:
 1.47201e-41 
 3.92903e-70 
 2.77759e-145

  0.254849 seconds (300.01 k allocations: 18.311 MiB, 78.50% gc time)


(100000,)

In [75]:
@time z2 = convertfloat64b(x, :LittleEndian)
display(z2[1:3])
display(z2[end-2:end])
size(z2)

3-element Array{Float64,1}:
  6.88627e-241
 -9.96849e-88 
  1.85544e-14 

3-element Array{Float64,1}:
 1.47201e-41 
 3.92903e-70 
 2.77759e-145

  0.011477 seconds (44 allocations: 3.817 MiB)


(100000,)